## DEMO

### 01 load model

In [1]:
import sys
sys.path.append("../../../deepyeast/")
import os

import numpy as np
from tqdm import trange
from skimage.io import imsave

from utils import load_model
from postprocess.post_process_utils import post_process_panoptic

In [3]:
model = load_model("../../../deepyeast_001/", "../../configs/config_wl.yaml", num_gpus=0)

restoring or initializing model...
restored model from ../../../deepyeast_001/ckpt-73000.
restored from checkpoint: ../../../deepyeast_001/ckpt-73000


### 02 load image and prediction mask

In [4]:
from nd2reader import ND2Reader
from analysis.utils import file_traverse

In [6]:
file_list = file_traverse("./test/001_EX01.nd2")
save_path = './test' 

In [7]:
for i in range(0, len(file_list)):
    file = file_list[i]
    basename = os.path.basename(file)[:-4]
    
    images=ND2Reader(file)
    img = np.array(images)
    predict = np.zeros(img.shape)
    print(basename)

    for f in trange(img.shape[0]):
        frame = img[f]
        output = model.predict(frame)
        mask = post_process_panoptic(output["panoptic_pred"][0].numpy(), area_threshold=300)
        predict[f] = mask
    imsave(os.path.join(save_path, basename+"_mask.tif"),predict)

001_EX01


100%|███████████████████████████████████████████████████████████| 11/11 [08:08<00:00, 44.40s/it]
